# Imports

## libraries

In [5]:
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.efficientnet_v2 import EfficientNetV2B0, preprocess_input
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from keras import Sequential, layers
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.regularizers import l2
from PIL import Image 
import matplotlib.pyplot as plt

2025-09-05 13:35:30.333282: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1757079330.549632      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1757079330.613768      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


## data

In [2]:
#import the data
df_train=pd.read_csv('/kaggle/input/mushroom1/train.csv')
df_val=pd.read_csv('/kaggle/input/mushroom1/val.csv')
df_test=pd.read_csv('/kaggle/input/mushroom1/test.csv')

# Preprocessing

## Functions

### fixing path

In [3]:
#creating a function to fix path
def fix_path(df) :
    df=df.drop_duplicates()
    df["fixed_path"] = df["image_path"].apply(
    lambda p: p.replace("/kaggle/working/merged_dataset", "/kaggle/input/mushroom1/merged_dataset")
    )
    return df

### creating generator

In [19]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
#creating a generator that transforms path to images batch size per batch size

def generator(df,x_col,y_col,img_size,batch_size,shuffle,data_augment) :

    if data_augment :
        datagen = ImageDataGenerator(rescale=1./255,
                                 rotation_range=20,
                                 width_shift_range=0.1,
                                 height_shift_range=0.1,
                                 zoom_range=0.2,
                                 horizontal_flip=True) 
    else :
        datagen = ImageDataGenerator(rescale=1./255)

    gen = datagen.flow_from_dataframe(
    dataframe=df,
    x_col=x_col,
    y_col=y_col,
    target_size=img_size,
    color_mode="rgb",
    class_mode="categorical",    
    batch_size=batch_size,
    shuffle=shuffle,
    )
    return gen



## Applying functions

In [5]:
#fixing path and dropping duplicates
df_train=fix_path(df_train)
df_val=fix_path(df_val)
df_test=fix_path(df_test)


/tmp/ipykernel_36/1883868460.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["fixed_path"] = df["image_path"].apply(


In [6]:
#using the generator for our params and df

#our params : 
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
X_COL="fixed_path"
Y_COL="label"

train_gen=generator(df_train,X_COL,Y_COL,IMG_SIZE,BATCH_SIZE,shuffle=True,data_augment=True)
test_gen=generator(df_test,X_COL,Y_COL,IMG_SIZE,BATCH_SIZE,shuffle=False,data_augment=False)
val_gen=generator(df_val,X_COL,Y_COL,IMG_SIZE,BATCH_SIZE,shuffle=False,data_augment=False)



Found 72858 validated image filenames belonging to 169 classes.
Found 15614 validated image filenames belonging to 169 classes.
Found 15616 validated image filenames belonging to 169 classes.


# Base Model

## Model architecture

### Transferlearning import

In [7]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers, models

IMG_SIZE = (224, 224)  # ViT-B/16 TF-Hub prend 224x224

# 1) Charge un feature-extractor ViT-B/16 (sans la tête)
vit_url = "https://tfhub.dev/sayakpaul/vit_b16_fe/1"  # feature extractor (FE = notop)
vit_layer = hub.KerasLayer(vit_url, trainable=False, name="vit_b16_fe")

I0000 00:00:1757063305.345640      36 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


### Architecture

In [8]:
model = Sequential()

model.add(layers.Input(shape=(224,224, 3)))

#using lambda so it works with sequential
model.add(layers.Lambda(lambda x: vit_layer(x, training=False), name="vit_fe"))
model.add(layers.Dropout(0.3))
model.add(layers.Dense(169, activation="softmax"))

In [9]:
xb, yb = next(iter(train_gen))
print(vit_layer(xb, training=False).shape)

I0000 00:00:1756902961.770864      36 service.cc:148] XLA service 0x4c5bc6d0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1756902961.771577      36 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1756902962.542404      36 cuda_dnn.cc:529] Loaded cuDNN version 90300


(32, 768)


I0000 00:00:1756902964.613156      36 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


In [10]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ vit_fe (Lambda)                 │ (None, 768)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 768)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 169)            │       129,961 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 129,961 (507.66 KB)

 Trainable params: 129,961 (507.66 KB)

 Non-trainable params: 0 (0.00 B)

## Compiling

In [9]:
def model_compiling(model) :
    #compiling
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy','precision','recall'])
    return model


In [10]:
model_compiling(model)

<Sequential name=sequential, built=True>

## Fitting

In [11]:
def model_fitting(model,train_data,validation_data,epochs=10) :

    #callbacks
    es = [EarlyStopping(patience=5, restore_best_weights=True, monitor="val_accuracy")]

    #fitting
    history = model.fit(
    train_data,
    validation_data=validation_data,
    epochs=epochs,
    verbose=1,
    callbacks=es
    )
    
    return history

In [14]:
history=model_fitting(model,train_gen,val_gen,epochs=5)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/5
2277/2277 ━━━━━━━━━━━━━━━━━━━━ 734s 317ms/step - accuracy: 0.5819 - loss: 1.9440 - precision: 0.7475 - recall: 0.5067 - val_accuracy: 0.8144 - val_loss: 0.6590 - val_precision: 0.8642 - val_recall: 0.7861
Epoch 2/5
2277/2277 ━━━━━━━━━━━━━━━━━━━━ 494s 217ms/step - accuracy: 0.8177 - loss: 0.6380 - precision: 0.8611 - recall: 0.7893 - val_accuracy: 0.8275 - val_loss: 0.6221 - val_precision: 0.8632 - val_recall: 0.8072
Epoch 3/5
2277/2277 ━━━━━━━━━━━━━━━━━━━━ 494s 217ms/step - accuracy: 0.8366 - loss: 0.5610 - precision: 0.8695 - recall: 0.8180 - val_accuracy: 0.8347 - val_loss: 0.6365 - val_precision: 0.8619 - val_recall: 0.8186
Epoch 4/5
2277/2277 ━━━━━━━━━━━━━━━━━━━━ 494s 217ms/step - accuracy: 0.8489 - loss: 0.5172 - precision: 0.8748 - recall: 0.8320 - val_accuracy: 0.8360 - val_loss: 0.6552 - val_precision: 0.8613 - val_recall: 0.8220
Epoch 5/5
2277/2277 ━━━━━━━━━━━━━━━━━━━━ 493s 216ms/step - accuracy: 0.8536 - loss: 0.5071 - precision: 0.8756 - recall: 0.8403 - val_accura

## Evaluating

In [13]:
def evaluate_model(model,test_data) :
    evaluation=model.evaluate(test_data)
    return evaluation

In [16]:
evaluate_model(model,test_gen)


488/488 ━━━━━━━━━━━━━━━━━━━━ 156s 320ms/step - accuracy: 0.8295 - loss: 0.6695 - precision: 0.8543 - recall: 0.8144


[0.667918860912323, 0.8320738077163696, 0.85657799243927, 0.8185602426528931]

## Saving the model

In [19]:
model.save("ViT_model.keras")
model.export("ViT_model.keras")

## Predicting 

In [46]:
def preprocess_for_predict(img_path) :
    img=Image.open(img_path)
    img=img.resize((224,224))
    arr=np.array(img).astype('float32')
    arr=arr/255.0
    arr.shape
    arr=np.expand_dims(arr,axis=0)
    return arr

In [47]:
def predict(model,img,index_to_class):
    #index_to_class={v :k for k,v in train_data.class_indices.items()}
    prediction=model.predict(img)
    index=np.argmax(prediction[0])
    proba=round(max(prediction[0])*100,2)
    mushroom=index_to_class.get(index)
    return mushroom,f"{proba}%"

In [ ]:
img=preprocess_for_predict('/kaggle/input/picture-test-crucibulum-laeve/image_test.jpg')
predict(model,img,train_gen)

### params for predicting

In [8]:
index_to_class={v :k for k,v in train_gen.class_indices.items()}

In [14]:
index_to_class={0: 'Agaricus augustus',
 1: 'Agaricus xanthodermus',
 2: 'Amanita amerirubescens',
 3: 'Amanita augusta',
 4: 'Amanita brunnescens',
 5: 'Amanita calyptroderma',
 6: 'Amanita citrina',
 7: 'Amanita flavoconia',
 8: 'Amanita muscaria',
 9: 'Amanita pantherina',
 10: 'Amanita persicina',
 11: 'Amanita phalloides',
 12: 'Amanita rubescens',
 13: 'Amanita velosa',
 14: 'Apioperdon pyriforme',
 15: 'Armillaria borealis',
 16: 'Armillaria mellea',
 17: 'Armillaria tabescens',
 18: 'Artomyces pyxidatus',
 19: 'Bjerkandera adusta',
 20: 'Bolbitius titubans',
 21: 'Boletus edulis',
 22: 'Boletus pallidus',
 23: 'Boletus reticulatus',
 24: 'Boletus rex-veris',
 25: 'Calocera viscosa',
 26: 'Calycina citrina',
 27: 'Cantharellus californicus',
 28: 'Cantharellus cibarius',
 29: 'Cantharellus cinnabarinus',
 30: 'Cerioporus squamosus',
 31: 'Cetraria islandica',
 32: 'Chlorociboria aeruginascens',
 33: 'Chlorophyllum brunneum',
 34: 'Chlorophyllum molybdites',
 35: 'Chondrostereum purpureum',
 36: 'Cladonia fimbriata',
 37: 'Cladonia rangiferina',
 38: 'Cladonia stellaris',
 39: 'Clitocybe nebularis',
 40: 'Clitocybe nuda',
 41: 'Coltricia perennis',
 42: 'Coprinellus disseminatus',
 43: 'Coprinellus micaceus',
 44: 'Coprinopsis atramentaria',
 45: 'Coprinopsis lagopus',
 46: 'Coprinus comatus',
 47: 'Crucibulum laeve',
 48: 'Cryptoporus volvatus',
 49: 'Daedaleopsis confragosa',
 50: 'Daedaleopsis tricolor',
 51: 'Entoloma abortivum',
 52: 'Evernia mesomorpha',
 53: 'Evernia prunastri',
 54: 'Flammulina velutipes',
 55: 'Fomes fomentarius',
 56: 'Fomitopsis betulina',
 57: 'Fomitopsis mounceae',
 58: 'Fomitopsis pinicola',
 59: 'Galerina marginata',
 60: 'Ganoderma applanatum',
 61: 'Ganoderma curtisii',
 62: 'Ganoderma oregonense',
 63: 'Ganoderma tsugae',
 64: 'Gliophorus psittacinus',
 65: 'Gloeophyllum sepiarium',
 66: 'Graphis scripta',
 67: 'Grifola frondosa',
 68: 'Gymnopilus luteofolius',
 69: 'Gyromitra esculenta',
 70: 'Gyromitra gigas',
 71: 'Gyromitra infula',
 72: 'Hericium coralloides',
 73: 'Hericium erinaceus',
 74: 'Hygrophoropsis aurantiaca',
 75: 'Hypholoma fasciculare',
 76: 'Hypholoma lateritium',
 77: 'Hypogymnia physodes',
 78: 'Hypomyces lactifluorum',
 79: 'Imleria badia',
 80: 'Inonotus obliquus',
 81: 'Ischnoderma resinosum',
 82: 'Kuehneromyces mutabilis',
 83: 'Laccaria ochropurpurea',
 84: 'Lactarius deliciosus',
 85: 'Lactarius torminosus',
 86: 'Lactarius turpis',
 87: 'Laetiporus sulphureus',
 88: 'Leccinum albostipitatum',
 89: 'Leccinum aurantiacum',
 90: 'Leccinum scabrum',
 91: 'Leccinum versipelle',
 92: 'Lepista nuda',
 93: 'Leratiomyces ceres',
 94: 'Leucoagaricus americanus',
 95: 'Leucoagaricus leucothites',
 96: 'Lobaria pulmonaria',
 97: 'Lycogala epidendrum',
 98: 'Lycoperdon perlatum',
 99: 'Lycoperdon pyriforme',
 100: 'Macrolepiota procera',
 101: 'Merulius tremellosus',
 102: 'Mutinus ravenelii',
 103: 'Mycena haematopus',
 104: 'Mycena leaiana',
 105: 'Nectria cinnabarina',
 106: 'Omphalotus illudens',
 107: 'Omphalotus olivascens',
 108: 'Panaeolus papilionaceus',
 109: 'Panellus stipticus',
 110: 'Parmelia sulcata',
 111: 'Paxillus involutus',
 112: 'Peltigera aphthosa',
 113: 'Peltigera praetextata',
 114: 'Phaeolus schweinitzii',
 115: 'Phaeophyscia orbicularis',
 116: 'Phallus impudicus',
 117: 'Phellinus igniarius',
 118: 'Phellinus tremulae',
 119: 'Phlebia radiata',
 120: 'Phlebia tremellosa',
 121: 'Pholiota aurivella',
 122: 'Pholiota squarrosa',
 123: 'Phyllotopsis nidulans',
 124: 'Physcia adscendens',
 125: 'Platismatia glauca',
 126: 'Pleurotus ostreatus',
 127: 'Pleurotus pulmonarius',
 128: 'Psathyrella candolleana',
 129: 'Pseudevernia furfuracea',
 130: 'Pseudohydnum gelatinosum',
 131: 'Psilocybe azurescens',
 132: 'Psilocybe caerulescens',
 133: 'Psilocybe cubensis',
 134: 'Psilocybe cyanescens',
 135: 'Psilocybe ovoideocystidiata',
 136: 'Psilocybe pelliculosa',
 137: 'Retiboletus ornatipes',
 138: 'Rhytisma acerinum',
 139: 'Sarcomyxa serotina',
 140: 'Sarcoscypha austriaca',
 141: 'Sarcosoma globosum',
 142: 'Schizophyllum commune',
 143: 'Stereum hirsutum',
 144: 'Stereum ostrea',
 145: 'Stropharia aeruginosa',
 146: 'Stropharia ambigua',
 147: 'Suillus americanus',
 148: 'Suillus granulatus',
 149: 'Suillus grevillei',
 150: 'Suillus luteus',
 151: 'Suillus spraguei',
 152: 'Tapinella atrotomentosa',
 153: 'Trametes betulina',
 154: 'Trametes gibbosa',
 155: 'Trametes hirsuta',
 156: 'Trametes ochracea',
 157: 'Trametes versicolor',
 158: 'Tremella mesenterica',
 159: 'Trichaptum biforme',
 160: 'Tricholoma murrillianum',
 161: 'Tricholomopsis rutilans',
 162: 'Tylopilus felleus',
 163: 'Tylopilus rubrobrunneus',
 164: 'Urnula craterium',
 165: 'Verpa bohemica',
 166: 'Volvopluteus gloiocephalus',
 167: 'Vulpicida pinastri',
 168: 'Xanthoria parietina'}

# Model Experimentation

## initialization

In [16]:
#our params : 
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
X_COL="fixed_path"
Y_COL="label"

#fixing path and dropping duplicates
df_train=fix_path(df_train)
df_val=fix_path(df_val)
df_test=fix_path(df_test)

#using the generator for our params and df
train_gen=generator(df_train,X_COL,Y_COL,IMG_SIZE,BATCH_SIZE,shuffle=True)
test_gen=generator(df_test,X_COL,Y_COL,IMG_SIZE,BATCH_SIZE,shuffle=False)
val_gen=generator(df_val,X_COL,Y_COL,IMG_SIZE,BATCH_SIZE,shuffle=False)


Found 72858 validated image filenames belonging to 169 classes.
Found 15614 validated image filenames belonging to 169 classes.
Found 15616 validated image filenames belonging to 169 classes.


In [18]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers, models

IMG_SIZE = (224, 224)  # ViT-B/16 TF-Hub prend 224x224

# 1) Charge un feature-extractor ViT-B/16 (sans la tête)
#vit_url = "https://tfhub.dev/sayakpaul/vit_b16_fe/1"  # feature extractor (FE = notop)
#vit_layer = hub.KerasLayer(vit_url, trainable=False, name="vit_b16_fe")
new_model = Sequential()

new_model.add(layers.Input(shape=(224,224, 3)))

#using lambda so it works with sequential
new_model.add(layers.Lambda(lambda x: vit_layer(x, training=False), name="vit_fe"))
new_model.add(layers.Dropout(0.3))
new_model.add(layers.Dense(169, activation="softmax"))

## Model compiling and fitting

In [13]:
model_compiling(model)
history=model_fitting(model,train_gen,val_gen,epochs=7)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/7


I0000 00:00:1757063342.848500     101 service.cc:148] XLA service 0x7f3f0c007560 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1757063342.849232     101 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1757063343.776474     101 cuda_dnn.cc:529] Loaded cuDNN version 90300


   1/2277 ━━━━━━━━━━━━━━━━━━━━ 8:06:48 13s/step - accuracy: 0.0000e+00 - loss: 8.2081 - precision: 0.0000e+00 - recall: 0.0000e+00

I0000 00:00:1757063346.664939     101 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


2277/2277 ━━━━━━━━━━━━━━━━━━━━ 1944s 849ms/step - accuracy: 0.5462 - loss: 2.1193 - precision: 0.7237 - recall: 0.4651 - val_accuracy: 0.7894 - val_loss: 0.7594 - val_precision: 0.8471 - val_recall: 0.7537
Epoch 2/7
2277/2277 ━━━━━━━━━━━━━━━━━━━━ 1226s 538ms/step - accuracy: 0.7745 - loss: 0.8252 - precision: 0.8310 - recall: 0.7399 - val_accuracy: 0.8060 - val_loss: 0.7216 - val_precision: 0.8521 - val_recall: 0.7816
Epoch 3/7
2277/2277 ━━━━━━━━━━━━━━━━━━━━ 1262s 554ms/step - accuracy: 0.7961 - loss: 0.7473 - precision: 0.8381 - recall: 0.7708 - val_accuracy: 0.8102 - val_loss: 0.7186 - val_precision: 0.8461 - val_recall: 0.7902
Epoch 4/7
2277/2277 ━━━━━━━━━━━━━━━━━━━━ 1295s 569ms/step - accuracy: 0.8060 - loss: 0.7153 - precision: 0.8400 - recall: 0.7851 - val_accuracy: 0.8092 - val_loss: 0.7370 - val_precision: 0.8428 - val_recall: 0.7918
Epoch 5/7
2277/2277 ━━━━━━━━━━━━━━━━━━━━ 1274s 560ms/step - accuracy: 0.8114 - loss: 0.7018 - precision: 0.8428 - recall: 0.7929 - val_accuracy: 0

In [20]:
val_gen=generator(df_val,X_COL,Y_COL,IMG_SIZE,BATCH_SIZE,shuffle=False,data_augment=False)
test_gen=generator(df_test,X_COL,Y_COL,IMG_SIZE,BATCH_SIZE,shuffle=False,data_augment=False)


Found 15616 validated image filenames belonging to 169 classes.
Found 15614 validated image filenames belonging to 169 classes.


NameError: name 'es' is not defined

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint

ckpt = ModelCheckpoint(
    "checkpoint.keras", save_best_only=False, save_weights_only=False
)
es = [EarlyStopping(patience=5, restore_best_weights=True, monitor="val_accuracy"),ckpt]
history_from_7=model.fit(train_gen,validation_data=val_gen,initial_epoch=8,epochs=10,callbacks=es)

Epoch 8/10
2277/2277 ━━━━━━━━━━━━━━━━━━━━ 0s 455ms/step - accuracy: 0.8165 - loss: 0.7139 - precision: 0.8414 - recall: 0.8019

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


2277/2277 ━━━━━━━━━━━━━━━━━━━━ 1136s 499ms/step - accuracy: 0.8165 - loss: 0.7139 - precision: 0.8414 - recall: 0.8019 - val_accuracy: 0.8358 - val_loss: 0.6947 - val_precision: 0.8585 - val_recall: 0.8246
Epoch 9/10
1113/2277 ━━━━━━━━━━━━━━━━━━━━ 8:50 456ms/step - accuracy: 0.8191 - loss: 0.6929 - precision: 0.8442 - recall: 0.8051

# evaluating

In [ ]:

evaluate_model(model,test_gen)

## Saving model

In [22]:
model.save("vit_saved_model.keras")
model.save("vit_saved_model.h5")
model.export("vit_saved_model")

## Loading and predicting functions

In [1]:
def load_vit_model(url="https://github.com/yves-rdlb/What-is-this-Mushroom/releases/download/vit_saved_model_v0/vit_saved_model.zip") :
    import os, zipfile, requests
    os.makedirs("models", exist_ok=True)
    zip_path = "models/vit_saved_model.zip"

    # download
    r = requests.get(url, stream=True)
    with open(zip_path, "wb") as f:
        for chunk in r.iter_content(8192):
            f.write(chunk)

    # extract
    with zipfile.ZipFile(zip_path, "r") as zip_ref:
        zip_ref.extractall("models")


    model_directory_path="models/vit_saved_model"
    reloaded=tf.saved_model.load(model_directory_path)
    infer=reloaded.signatures['serving_default']
    return infer

In [2]:
def preprocess_for_predict(img_path) :
    img=Image.open(img_path)
    img=img.resize((224,224))
    arr=np.array(img).astype('float32')
    arr=arr/255.0
    arr.shape
    arr=np.expand_dims(arr,axis=0)
    arr=tf.constant(arr,dtype=tf.float32)
    return arr


In [3]:
def predict(infer,preprocessed_img,index_to_class):
    #index_to_class={v :k for k,v in train_data.class_indices.items()}
    y=infer(preprocessed_img)
    prediction=(list(y.values())[0].numpy()[0])
    index=np.argmax(prediction)
    proba=round(max(prediction)*100,2)
    mushroom=index_to_class.get(index)
    return mushroom,f"{proba}%"

# Predicting and testing model

In [7]:
import tensorflow as tf
infer=load_vit_model()
preprocessed_img=preprocess_for_predict('/kaggle/input/picture-test-crucibulum-laeve/image_test.jpg')
predict(infer,preprocessed_img,index_to_class)

2025-09-05 13:37:14.941308: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


NameError: name 'index_to_class' is not defined